In [ ]:
import json
import warnings

warnings.filterwarnings(action='ignore')
import requests
import folium
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import numpy as np  # 수치계산, 배열 처리를 위해 import 한다.
import pandas as pd  # 데이터 분석을 위한 전처리를 위해 import 한다.
import matplotlib as mpl  # matplolib에서 음수 데이터의 '-'부호가 깨지는 것을 방지하기위해 import 한다.

mpl.rcParams['axes.unicode_minus'] = False  # matplolib에서 음수 데이터의 '-'부호가 깨지는 것을 방지한다.
import matplotlib.pyplot as plt  # 시각화를 하기 위해 import 한다.

plt.rcParams["font.size"] = 10  # matplolib에서 사용할 글꼴 크기 설정
plt.rcParams["font.family"] = "NanumGothicCoding"  # matplolib에서 사용할 글꼴 설정
import seaborn as sns  # 시각화를 하기 위해 import 한다. matplolib 기반으로 만든 시각화 도구

pd.options.display.float_format = '{:,.5f}'.format
from plotnine import *  # R의 ggplot2 패키지를 사용하는 것 처럼 사용하는 시각화 도구를 import 한다.
import missingno as msno  # 누락값의 분포를 그래프 형태로 보기위해 import 한다.
import folium  # 지도 시각화를 하기 위해 import 한다.
from wordcloud import WordCloud

In [ ]:
api_key = ''
headers = {'x-api-key': api_key}
url = 'https://open-api.bser.io/v1/'

In [ ]:
req = requests.get(url + 'user/nickname?query=Anbin', headers=headers)  # 닉네임으로 uid 호출
print(req.text)
time.sleep(1)

In [41]:
req2 = requests.get(url + 'user/stats/1072467/13',
                    headers=headers)  # uid/season 으로 유저의 해당 시즌 통계 호출 <구하려는 시즌*2-1  (현재시즌 8)>
time.sleep(1)
req2.text

'{"code": 200, "message": "Success", "userStats": [{"seasonId": 13, "userNum": 1072467, "matchingMode": 3, "matchingTeamMode": 2, "mmr": 1355, "nickname": "Anbin", "rank": 9807, "rankSize": 28094, "totalGames": 13, "totalWins": 3, "totalTeamKills": 69, "escapeCount": 0, "rankPercent": 0.35, "averageRank": 3.69, "averageKills": 2.31, "averageAssistants": 2.69, "averageHunts": 18.23, "top1": 0.23, "top2": 0.38, "top3": 0.46, "top5": 0.77, "top7": 0.92, "characterStats": [{"characterCode": 20, "totalGames": 10, "usages": 10, "maxKillings": 6, "top3": 5, "wins": 2, "top3Rate": 0, "averageRank": 4}, {"characterCode": 45, "totalGames": 2, "usages": 2, "maxKillings": 1, "top3": 0, "wins": 0, "top3Rate": 0, "averageRank": 5}, {"characterCode": 21, "totalGames": 1, "usages": 1, "maxKillings": 6, "top3": 1, "wins": 1, "top3Rate": 1, "averageRank": 1}]}, {"seasonId": 13, "userNum": 1072467, "matchingMode": 3, "matchingTeamMode": 3, "mmr": 2581, "nickname": "Anbin", "rank": 604, "rankSize": 26005,

In [42]:
test = requests.get(url + 'user/stats/725291/13', headers=headers).json()
test = json_normalize(test, 'userStats')
test[(test.matchingTeamMode == 3) & (test.matchingMode == 3)]

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
2,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1.00000,"[{'characterCode': 43, 'totalGames': 411, 'usa..."


In [43]:
req3 = requests.get(url + 'user/games/1072467', headers=headers)  # uid로 최근 90게임 전적 검색
time.sleep(1)
req3.text

'{"code": 200, "message": "Success", "userGames": [{"userNum": 1072467, "nickname": "Anbin", "gameId": 22654859, "seasonId": 0, "matchingMode": 6, "matchingTeamMode": 4, "characterNum": 58, "skinCode": 1058001, "characterLevel": 19, "gameRank": 1, "playerKill": 5, "playerAssistant": 15, "monsterKill": 21, "bestWeapon": 10, "bestWeaponLevel": 18, "masteryLevel": {"10": 18, "204": 6, "102": 6, "103": 6, "104": 6, "201": 6, "202": 15, "101": 6}, "equipment": {"0": 117502, "1": 202419, "2": 201411, "3": 203306, "4": 204413, "5": 705608}, "versionMajor": 75, "versionMinor": 0, "language": "Korean", "skillLevelInfo": {"1058200": 5, "1058300": 3, "1058400": 5, "1058500": 2, "1058100": 2, "3010000": 2, "1058510": 1}, "skillOrderInfo": {"1": 1058200, "2": 1058300, "3": 1058400, "4": 1058500, "5": 1058200, "6": 1058200, "7": 1058200, "8": 1058200, "9": 1058100, "10": 3010000, "11": 1058500, "12": 1058100, "13": 1058400, "14": 1058400, "15": 1058400, "16": 3010000, "17": 1058510, "18": 1058400, "

In [44]:
top1000 = requests.get(url + 'rank/top/13/3', headers=headers)  # 상위 1001명 정보 호출
print(top1000.text)
time.sleep(1)

{"code": 200, "message": "Success", "topRanks": [{"userNum": 725291, "nickname": "셀쓰녀가취향", "rank": 1, "mmr": 5208, "userEmblems": []}, {"userNum": 207458, "nickname": "SOS", "rank": 2, "mmr": 5203, "userEmblems": []}, {"userNum": 576995, "nickname": "왕왕이", "rank": 3, "mmr": 4954, "userEmblems": []}, {"userNum": 542829, "nickname": "갓루디", "rank": 4, "mmr": 4885, "userEmblems": []}, {"userNum": 570749, "nickname": "ジュルル", "rank": 5, "mmr": 4803, "userEmblems": []}, {"userNum": 463952, "nickname": "데굴데굴미숫가루", "rank": 6, "mmr": 4741, "userEmblems": []}, {"userNum": 1553415, "nickname": "적을찾는피5라", "rank": 7, "mmr": 4723, "userEmblems": []}, {"userNum": 360435, "nickname": "게임적응못한사람", "rank": 8, "mmr": 4655, "userEmblems": []}, {"userNum": 429054, "nickname": "リルパ", "rank": 9, "mmr": 4631, "userEmblems": []}, {"userNum": 2854562, "nickname": "민트초코와플", "rank": 10, "mmr": 4614, "userEmblems": []}, {"userNum": 501191, "nickname": "말파", "rank": 11, "mmr": 4602, "userEmblems": []}, {"userNum": 29

In [45]:
print(type(top1000))

<class 'requests.models.Response'>


In [46]:
top1000_json = top1000.json()
print(type(top1000_json))
top1000_json

<class 'dict'>


{'code': 200,
 'message': 'Success',
 'topRanks': [{'userNum': 725291,
   'nickname': '셀쓰녀가취향',
   'rank': 1,
   'mmr': 5208,
   'userEmblems': []},
  {'userNum': 207458,
   'nickname': 'SOS',
   'rank': 2,
   'mmr': 5203,
   'userEmblems': []},
  {'userNum': 576995,
   'nickname': '왕왕이',
   'rank': 3,
   'mmr': 4954,
   'userEmblems': []},
  {'userNum': 542829,
   'nickname': '갓루디',
   'rank': 4,
   'mmr': 4885,
   'userEmblems': []},
  {'userNum': 570749,
   'nickname': 'ジュルル',
   'rank': 5,
   'mmr': 4803,
   'userEmblems': []},
  {'userNum': 463952,
   'nickname': '데굴데굴미숫가루',
   'rank': 6,
   'mmr': 4741,
   'userEmblems': []},
  {'userNum': 1553415,
   'nickname': '적을찾는피5라',
   'rank': 7,
   'mmr': 4723,
   'userEmblems': []},
  {'userNum': 360435,
   'nickname': '게임적응못한사람',
   'rank': 8,
   'mmr': 4655,
   'userEmblems': []},
  {'userNum': 429054,
   'nickname': 'リルパ',
   'rank': 9,
   'mmr': 4631,
   'userEmblems': []},
  {'userNum': 2854562,
   'nickname': '민트초코와플',
   'rank': 

In [47]:
topRanks = json_normalize(top1000_json, 'topRanks')
topRanks

,userNum,nickname,rank,mmr,userEmblems
0,725291,셀쓰녀가취향,1,5208,[]
1,207458,SOS,2,5203,[]
2,576995,왕왕이,3,4954,[]
3,542829,갓루디,4,4885,[]
4,570749,ジュルル,5,4803,[]
...,...,...,...,...,...
996,1228334,우숙이,997,2556,[]
997,2144698,冷夜冷某人,998,2556,[]
998,1221355,반찬투정하는동기,999,2555,[]
999,3319648,くるみ,1000,2554,[]


In [48]:
print(type(top1000_json))
print(type(topRanks))

<class 'dict'>
<class 'pandas.core.frame.DataFrame'>


In [49]:
ranker = []  # top1000 uid만 별도로 리스트화
for index, i in topRanks.iterrows():
    # print(f'{topRanks.userNum[i]}')
    ranker.append(topRanks.userNum[index])

In [50]:
print(ranker)

[725291, 207458, 576995, 542829, 570749, 463952, 1553415, 360435, 429054, 2854562, 501191, 294507, 1374507, 3515612, 224287, 572087, 478304, 328575, 218512, 1193238, 3181361, 2952493, 866258, 1936623, 604477, 765136, 3668214, 3167172, 268108, 1393502, 103259, 2851871, 3307720, 693091, 3443475, 2482060, 246974, 848720, 379734, 412738, 3709033, 875248, 40020, 1151134, 108708, 1161005, 2952712, 563813, 1160179, 3702021, 702493, 1250073, 1534874, 384505, 1306380, 1466937, 3031115, 666750, 3772418, 228728, 2120261, 70352, 1484744, 3654679, 2355705, 91414, 1279055, 2190193, 2754248, 2762500, 209353, 2277240, 591743, 45029, 293844, 1167767, 767069, 1651844, 2960633, 1212122, 828577, 3560430, 3890017, 223570, 935564, 293551, 3466750, 300279, 112631, 2230466, 760221, 772291, 549148, 690711, 1962748, 276909, 1199780, 3202997, 1173974, 2764600, 658848, 2491651, 1383248, 474998, 116371, 93340, 827895, 656493, 3310706, 2374732, 2527977, 3646394, 2129859, 58060, 115973, 606681, 1106350, 44146, 10995

In [51]:
df = pd.DataFrame()
req = requests.get(url + 'user/stats/1072467/13', headers=headers)
test = (json_normalize(req.json(), 'userStats'))

# df.append(test[(test.matchingMode==3)&(test.matchingTeamMode==3)])

df2 = test[(test.matchingMode == 3) & (test.matchingTeamMode == 2)]
df3 = df.append(df2, ignore_index=True)
df = df.append(df3)
df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,1072467,3,2,1355,Anbin,9807,28094,13,3,...,3.69000,2.31000,2.69000,18.23000,0.23000,0.38000,0.46000,0.77000,0.92000,"[{'characterCode': 20, 'totalGames': 10, 'usag..."


In [52]:
'''     top 1000 유저 통계(스쿼드만) 데이터프레임화
topRanks_df = pd.DataFrame()
for index,uid in topRanks.iterrows():
    req=requests.get(url+f'user/stats/{uid[0]}/13', headers=headers)
    info = (json_normalize(req.json(),'userStats'))
    info_fin = info[(info.matchingMode==3)&(info.matchingTeamMode==3)]
    time.sleep(1)
    # print(f'{topRanks[topRanks.userNum==info_fin.userNum.iloc[0]]["rank"].iloc[0]}등: {info_fin.nickname.iloc[0]} MMR: {info_fin.mmr.iloc[0]}')
    print(f'{index+1}등: {info_fin.nickname.iloc[0]} MMR: {info_fin.mmr.iloc[0]}')

    topRanks_df = pd.concat([topRanks_df,info_fin], ignore_index=True)
topRanks_df
topRanks_df.drop(1000, inplace=True)
topRanks_df.to_csv('out/topRanks_df.csv')
'''
pass

In [53]:
topRanks_df = pd.read_csv('data/topRanks_df - changed.csv')

In [54]:
topRanks_df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1,"[{'characterCode': 43, 'totalGames': 411, 'usa..."
1,13,207458,3,3,5203,SOS,2,25384,697,369,...,2.31000,4.04000,3.32000,26.75000,0.53000,0.64000,0.74000,0.94000,1,"[{'characterCode': 1, 'totalGames': 510, 'usag..."
2,13,576995,3,3,4954,왕왕이,3,31102,568,279,...,2.44000,2.68000,5.93000,17.77000,0.49000,0.62000,0.71000,0.92000,1,"[{'characterCode': 17, 'totalGames': 567, 'usa..."
3,13,542829,3,3,4885,갓루디,4,30945,844,386,...,2.39000,2.06000,4.53000,16.45000,0.46000,0.61000,0.73000,0.96000,1,"[{'characterCode': 51, 'totalGames': 427, 'usa..."
4,13,570749,3,3,4803,ジュルル,5,33022,449,196,...,2.49000,3.96000,3.55000,25.68000,0.44000,0.58000,0.71000,0.94000,1,"[{'characterCode': 36, 'totalGames': 117, 'usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13,448264,3,3,2556,메갈리안,994,30465,99,23,...,3.22000,1.89000,3.11000,11.75000,0.23000,0.38000,0.56000,0.87000,1,"[{'characterCode': 28, 'totalGames': 30, 'usag..."
996,13,1228334,3,3,2556,우숙이,994,29099,127,34,...,3.13000,3.31000,3.28000,17.12000,0.27000,0.42000,0.57000,0.88000,1,"[{'characterCode': 9, 'totalGames': 109, 'usag..."
997,13,2144698,3,3,2556,冷夜冷某人,994,26189,235,51,...,3.40000,0.96000,3.79000,7.71000,0.22000,0.37000,0.51000,0.83000,1,"[{'characterCode': 41, 'totalGames': 135, 'usa..."
998,13,1221355,3,3,2555,반찬투정하는동기,999,26977,345,34,...,3.53000,1.54000,2.22000,24.96000,0.10000,0.28000,0.51000,0.88000,1,"[{'characterCode': 52, 'totalGames': 171, 'usa..."


In [55]:
'''             top 1000 유저 most3 데이터프레임화
realRank=1
virRank=1
j=topRanks_df.mmr[0]
for index,i in topRanks_df.iterrows():
    if(i.mmr==j):
        topRanks_df.loc[index,'rank']=realRank
        virRank+=1
        j=i.mmr
    else:
        realRank=virRank
        topRanks_df.loc[index,'rank']=realRank
        realRank+=1
        virRank+=1
        j=i.mmr
topRanks_df
topRanks_df.to_csv('out/topRanks_df.csv')
'''
pass

In [56]:
print(topRanks_df)

     seasonId  userNum  matchingMode  matchingTeamMode   mmr  nickname  rank  \
0          13   725291             3                 3  5208    셀쓰녀가취향     1   
1          13   207458             3                 3  5203       SOS     2   
2          13   576995             3                 3  4954       왕왕이     3   
3          13   542829             3                 3  4885       갓루디     4   
4          13   570749             3                 3  4803      ジュルル     5   
..        ...      ...           ...               ...   ...       ...   ...   
995        13   448264             3                 3  2556      메갈리안   994   
996        13  1228334             3                 3  2556       우숙이   994   
997        13  2144698             3                 3  2556     冷夜冷某人   994   
998        13  1221355             3                 3  2555  반찬투정하는동기   999   
999        13  3319648             3                 3  2554       くるみ  1000   

     rankSize  totalGames  totalWins  .

In [57]:
'''             캐릭터 코드 => 캐릭터 한글 이름
characterList = requests.get(url + 'data/Character', headers=headers)
characterList = characterList.json()
time.sleep(1)
characterList = json_normalize(characterList, 'data')
characterList = characterList[['code', 'name']]
characterList.to_csv('out/characterList.csv')
'''
pass

In [58]:
characterList = pd.read_csv('data/characterList - changed.csv')
# characterList.set_index(pd.Index(range(1, 63)), inplace=True)
characterList.set_index('code', inplace=True)

In [130]:
characterList

,name
code,
1,재키
2,아야
3,피오라
4,매그너스
5,자히르
...,...
58,헤이즈
59,아이작
60,타지아


In [131]:
topRanks_df

,seasonId,userNum,matchingMode,matchingTeamMode,mmr,nickname,rank,rankSize,totalGames,totalWins,...,averageRank,averageKills,averageAssistants,averageHunts,top1,top2,top3,top5,top7,characterStats
0,13,725291,3,3,5208,셀쓰녀가취향,1,30457,802,436,...,2.34000,3.71000,3.83000,26.56000,0.54000,0.63000,0.73000,0.93000,1,"[{'characterCode': 43, 'totalGames': 411, 'usa..."
1,13,207458,3,3,5203,SOS,2,25384,697,369,...,2.31000,4.04000,3.32000,26.75000,0.53000,0.64000,0.74000,0.94000,1,"[{'characterCode': 1, 'totalGames': 510, 'usag..."
2,13,576995,3,3,4954,왕왕이,3,31102,568,279,...,2.44000,2.68000,5.93000,17.77000,0.49000,0.62000,0.71000,0.92000,1,"[{'characterCode': 17, 'totalGames': 567, 'usa..."
3,13,542829,3,3,4885,갓루디,4,30945,844,386,...,2.39000,2.06000,4.53000,16.45000,0.46000,0.61000,0.73000,0.96000,1,"[{'characterCode': 51, 'totalGames': 427, 'usa..."
4,13,570749,3,3,4803,ジュルル,5,33022,449,196,...,2.49000,3.96000,3.55000,25.68000,0.44000,0.58000,0.71000,0.94000,1,"[{'characterCode': 36, 'totalGames': 117, 'usa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,13,448264,3,3,2556,메갈리안,994,30465,99,23,...,3.22000,1.89000,3.11000,11.75000,0.23000,0.38000,0.56000,0.87000,1,"[{'characterCode': 28, 'totalGames': 30, 'usag..."
996,13,1228334,3,3,2556,우숙이,994,29099,127,34,...,3.13000,3.31000,3.28000,17.12000,0.27000,0.42000,0.57000,0.88000,1,"[{'characterCode': 9, 'totalGames': 109, 'usag..."
997,13,2144698,3,3,2556,冷夜冷某人,994,26189,235,51,...,3.40000,0.96000,3.79000,7.71000,0.22000,0.37000,0.51000,0.83000,1,"[{'characterCode': 41, 'totalGames': 135, 'usa..."
998,13,1221355,3,3,2555,반찬투정하는동기,999,26977,345,34,...,3.53000,1.54000,2.22000,24.96000,0.10000,0.28000,0.51000,0.88000,1,"[{'characterCode': 52, 'totalGames': 171, 'usa..."


In [61]:
test = topRanks_df.characterStats[2].replace("\'", "\"")
test = {'characterStats': json.loads(test)}
print(test['characterStats'][0])
test_json = {'most': test['characterStats'][0]}
print(characterList.name[test['characterStats'][0]['characterCode']])
print(test.values())

{'characterCode': 17, 'totalGames': 567, 'usages': 567, 'maxKillings': 10, 'top3': 406, 'wins': 279, 'top3Rate': 1, 'averageRank': 2}
아드리아나
dict_values([[{'characterCode': 17, 'totalGames': 567, 'usages': 567, 'maxKillings': 10, 'top3': 406, 'wins': 279, 'top3Rate': 1, 'averageRank': 2}, {'characterCode': 36, 'totalGames': 1, 'usages': 1, 'maxKillings': 0, 'top3': 0, 'wins': 0, 'top3Rate': 0, 'averageRank': 5}]])


In [62]:
for index, i in topRanks_df.iterrows():
    characterStats = i.characterStats.replace("\'", "\"")  # df 안에 리스트가 '[]' 상태여서 json 변환을 위해 '와 " replace함
    characterStats_json = {'characterStats': json.loads(characterStats)}
    stats = characterStats_json['characterStats']
    try:
        if stats[2] and stats[1] and stats[0]:
            print(f'{topRanks.nickname[index]}')
            print('most1: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                                             (stats[0]['totalGames']),
                                                             (stats[0]['wins'] / stats[0]['totalGames'] * 100)))
            print('most2: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[1]['characterCode']],
                                                             (stats[1]['totalGames']),
                                                             (stats[1]['wins'] / stats[1]['totalGames'] * 100)))
            print('most3: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[2]['characterCode']],
                                                             (stats[2]['totalGames']),
                                                             (stats[2]['wins'] / stats[2]['totalGames'] * 100)))
    except:
        try:
            print(f'{topRanks.nickname[index]}')
            if stats[1] and stats[0]:
                print('most1: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                                                 (stats[0]['totalGames']),
                                                                 (stats[0]['wins'] / stats[0]['totalGames'] * 100)))
                print('most2: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[1]['characterCode']],
                                                                 (stats[1]['totalGames']),
                                                                 (stats[1]['wins'] / stats[1]['totalGames'] * 100)))
        except:
            if stats[0]:
                print(f'{topRanks.nickname[index]}')
                print('most1: {}, playTimes: {}, winrate: {:.3f}'.format(characterList.name[stats[0]['characterCode']],
                                                                 (stats[0]['totalGames']),
                                                                 (stats[0]['wins'] / stats[0]['totalGames'] * 100)))

셀쓰녀가취향
most1: 셀린, playTimes: 411, winrate: 60.341
most2: 카밀로, playTimes: 336, winrate: 52.381
most3: 마르티나, playTimes: 23, winrate: 30.435
SOS
most1: 재키, playTimes: 510, winrate: 54.706
most2: 마커스, playTimes: 86, winrate: 60.465
most3: 현우, playTimes: 53, winrate: 43.396
왕왕이
most1: 아드리아나, playTimes: 567, winrate: 49.206
most2: 이바, playTimes: 1, winrate: 0.000
갓루디
most1: 프리야, playTimes: 427, winrate: 46.838
most2: 리다이린, playTimes: 398, winrate: 45.477
most3: 띠아, playTimes: 18, winrate: 27.778
ジュルル
most1: 이바, playTimes: 117, winrate: 48.718
most2: 재키, playTimes: 79, winrate: 40.506
most3: 혜진, playTimes: 59, winrate: 67.797
데굴데굴미숫가루
most1: 에이든, playTimes: 190, winrate: 51.579
most2: 얀, playTimes: 44, winrate: 45.455
most3: 실비아, playTimes: 42, winrate: 52.381
적을찾는피5라
most1: 하트, playTimes: 249, winrate: 53.012
most2: 혜진, playTimes: 35, winrate: 42.857
most3: 피오라, playTimes: 32, winrate: 40.625
게임적응못한사람
most1: 비앙카, playTimes: 87, winrate: 51.724
most2: 요한, playTimes: 46, winrate: 50.000
most3:

""


In [63]:
for index, i in topRanks_df.iterrows():
    characterStats = i.characterStats.replace("\'", "\"")  # df 안에 리스트가 '[]' 상태여서 json 변환을 위해 '와 " replace함
    characterStats_json = {'characterStats': json.loads(characterStats)}
    stats = characterStats_json['characterStats']
    if len(stats) >= 3:
            print(f'most : [{"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[0]["characterCode"]]}", "playTimes" : {stats[0]["totalGames"]}, "winrate" : {stats[0]["wins"] / stats[0]["totalGames"] * 100:.3f}{"}"}, {"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[1]["characterCode"]]}", "playTimes" : {stats[1]["totalGames"]}, "winrate" : {stats[1]["wins"] / stats[1]["totalGames"] * 100:.3f}{"}"}, {"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[2]["characterCode"]]}", "playTimes" : {stats[2]["totalGames"]}, "winrate" : {stats[2]["wins"] / stats[2]["totalGames"] * 100:.3f}{"}"}]')
            print()
    elif len(stats) >= 2:
                print(f'most : [{"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[0]["characterCode"]]}", "playTimes" : {stats[0]["totalGames"]}, "winrate" : {stats[0]["wins"] / stats[0]["totalGames"] * 100:.3f}{"}"}, {"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[1]["characterCode"]]}", "playTimes" : {stats[1]["totalGames"]}]')
                print()
    else:
                print(f'most : [{"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[stats[0]["characterCode"]]}", "playTimes" : {stats[0]["totalGames"]}, "winrate" : {stats[0]["wins"] / stats[0]["totalGames"] * 100:.3f}{"}"}]')
                print()

most : [{"name" : "셀쓰녀가취향","character" : "셀린", "playTimes" : 411, "winrate" : 60.341}, {"name" : "셀쓰녀가취향","character" : "카밀로", "playTimes" : 336, "winrate" : 52.381}, {"name" : "셀쓰녀가취향","character" : "마르티나", "playTimes" : 23, "winrate" : 30.435}]

most : [{"name" : "SOS","character" : "재키", "playTimes" : 510, "winrate" : 54.706}, {"name" : "SOS","character" : "마커스", "playTimes" : 86, "winrate" : 60.465}, {"name" : "SOS","character" : "현우", "playTimes" : 53, "winrate" : 43.396}]

most : [{"name" : "왕왕이","character" : "아드리아나", "playTimes" : 567, "winrate" : 49.206}, {"name" : "왕왕이","character" : "이바", "playTimes" : 1]

most : [{"name" : "갓루디","character" : "프리야", "playTimes" : 427, "winrate" : 46.838}, {"name" : "갓루디","character" : "리다이린", "playTimes" : 398, "winrate" : 45.477}, {"name" : "갓루디","character" : "띠아", "playTimes" : 18, "winrate" : 27.778}]

most : [{"name" : "ジュルル","character" : "이바", "playTimes" : 117, "winrate" : 48.718}, {"name" : "ジュルル","character" : "재키", "playTimes" : 

In [64]:
mostCharacter = pd.DataFrame()
for index, i in topRanks_df.iterrows():
    characterStats = i.characterStats.replace("\'", "\"")  # df 안에 리스트가 '[]' 상태여서 json 변환을 위해 '와 " replace함
    characterStats_json = {'characterStats': json.loads(characterStats)}
    stats = characterStats_json['characterStats']
    for j, most in enumerate(stats[:3]):
        # print(f'most : [{"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[most["characterCode"]]}", "playTimes" : {most["totalGames"]}, "winrate" : {most["wins"] / most["totalGames"] * 100:.3f}{"}"}]')
        # mc = f'{"{"}"most{j+1}" : {"{"}"name" : "{topRanks.nickname[index]}","character" : "{characterList.name[most["characterCode"]]}", "playTimes" : {most["totalGames"]}, "winrate" : {most["wins"] / most["totalGames"] * 100:.3f}{"}"}{"}"}'
        mc = json.loads(f'{"{"}"userNum" : "{topRanks.userNum[index]}", "nickname" : "{topRanks.nickname[index]}", "most" : {j+1}, "character" : "{characterList.name[most["characterCode"]]}", "playTimes" : {most["totalGames"]}, "winrate" : {most["wins"] / most["totalGames"] * 100:.3f}{"}"}')
        mc2 = json_normalize(mc)
        mostCharacter=pd.concat([mostCharacter,mc2], ignore_index=True)
mostCharacter

,userNum,nickname,most,character,playTimes,winrate
0,725291,셀쓰녀가취향,1,셀린,411,60.34100
1,725291,셀쓰녀가취향,2,카밀로,336,52.38100
2,725291,셀쓰녀가취향,3,마르티나,23,30.43500
3,207458,SOS,1,재키,510,54.70600
4,207458,SOS,2,마커스,86,60.46500
...,...,...,...,...,...,...
2841,1221355,반찬투정하는동기,2,나딘,26,15.38500
2842,1221355,반찬투정하는동기,3,띠아,22,9.09100
2843,3319648,くるみ,1,클로에,46,63.04300
2844,3319648,くるみ,2,하트,4,75.00000


In [66]:
mostCharacter[mostCharacter.nickname=='셀쓰녀가취향'].loc[0].character

'셀린'

In [67]:
'''             top 1000 유저 통계(솔로, 듀오, 스쿼드) 데이터프레임화
rankerStats_df = pd.DataFrame()
for i in ranker:
    rankerStats = requests.get(url + f'user/stats/{i}/13', headers=headers)
    rankerStats2 = json_normalize(rankerStats.json(), 'userStats')
    rankerStats_df = pd.concat([rankerStats_df, rankerStats2], ignore_index=True)
    time.sleep(1)
    print(rankerStats2['characterStats'])
rankerStats_df.to_csv('out/rankerStats_df.csv')
print('fin')
'''
pass

In [68]:
top1000Now = requests.get(url + 'rank/top/15/3', headers=headers)
time.sleep(1)


In [69]:
top1000Now_json = top1000Now.json()
print(type(top1000Now_json))
topRanksNow = json_normalize(top1000Now_json, 'topRanks')
topRanksNow

<class 'dict'>


,userNum,nickname,rank,mmr,userEmblems
0,725291,셀쓰녀가취향,1,2848,[]
1,1199780,탐피,2,2782,[]
2,2120261,태리프,3,2687,[]
3,478304,ASeuNa,4,2672,[]
4,576995,왕왕이,5,2669,[]
...,...,...,...,...,...
996,3331128,欣特莱娅,997,1599,[]
997,3331124,呆鹿,998,1599,[]
998,415840,아서라,999,1599,[]
999,1371471,你贡品保不住了,1000,1599,[]


In [70]:
rankerNow = []
for index, i in topRanksNow.iterrows():
    rankerNow.append(topRanksNow.userNum[index])
rankerNow


[725291,
 1199780,
 2120261,
 478304,
 576995,
 207458,
 570749,
 412738,
 772427,
 116371,
 268108,
 606681,
 69146,
 91905,
 293844,
 128411,
 469677,
 77166,
 1351282,
 687440,
 1197081,
 501191,
 70587,
 429054,
 765136,
 1534874,
 3305407,
 2762500,
 787397,
 2742624,
 1754989,
 1313406,
 422956,
 848720,
 3917968,
 869052,
 509796,
 1306380,
 623629,
 150147,
 572087,
 866258,
 44146,
 282448,
 1774931,
 544622,
 207703,
 3262941,
 463952,
 2775323,
 445720,
 102256,
 839617,
 2754248,
 98602,
 360435,
 104477,
 2960633,
 607913,
 1942800,
 70352,
 737274,
 36441,
 3668214,
 91414,
 674629,
 873854,
 224664,
 3466750,
 448994,
 394892,
 474194,
 1497010,
 59897,
 126501,
 168159,
 1651844,
 1193238,
 3844176,
 2690772,
 286088,
 2031716,
 626620,
 1298857,
 589764,
 218895,
 1720597,
 2230466,
 223570,
 2133501,
 563813,
 452355,
 1058273,
 255106,
 446157,
 157674,
 1212122,
 666750,
 3013783,
 3290647,
 2396832,
 405975,
 2005156,
 2854594,
 1936623,
 403491,
 3560430,
 967511,

In [71]:
now90 = requests.get(url + 'user/games/1072467', headers=headers)  # uid로 최근 90게임 전적 검색
time.sleep(1)
now90.text

'{"code": 200, "message": "Success", "userGames": [{"userNum": 1072467, "nickname": "Anbin", "gameId": 22654859, "seasonId": 0, "matchingMode": 6, "matchingTeamMode": 4, "characterNum": 58, "skinCode": 1058001, "characterLevel": 19, "gameRank": 1, "playerKill": 5, "playerAssistant": 15, "monsterKill": 21, "bestWeapon": 10, "bestWeaponLevel": 18, "masteryLevel": {"10": 18, "204": 6, "102": 6, "103": 6, "104": 6, "201": 6, "202": 15, "101": 6}, "equipment": {"0": 117502, "1": 202419, "2": 201411, "3": 203306, "4": 204413, "5": 705608}, "versionMajor": 75, "versionMinor": 0, "language": "Korean", "skillLevelInfo": {"1058200": 5, "1058300": 3, "1058400": 5, "1058500": 2, "1058100": 2, "3010000": 2, "1058510": 1}, "skillOrderInfo": {"1": 1058200, "2": 1058300, "3": 1058400, "4": 1058500, "5": 1058200, "6": 1058200, "7": 1058200, "8": 1058200, "9": 1058100, "10": 3010000, "11": 1058500, "12": 1058100, "13": 1058400, "14": 1058400, "15": 1058400, "16": 3010000, "17": 1058510, "18": 1058400, "

In [72]:
''' # 상위 유저들의 최근 90게임 통계
topRanksNow90_df = pd.DataFrame()
for index,uid in topRanksNow.iterrows():
    req=requests.get(url+f'user/games/{uid[0]}', headers=headers)
    info = (json_normalize(req.json(),'userGames'))
    info_fin = info[(info.matchingMode==3)&(info.matchingTeamMode==3)]
    print(f'{info_fin.nickname} ')
    topRanksNow90_df = pd.concat([topRanksNow90_df,info_fin], ignore_index=True)
    time.sleep(1)

topRanksNow90_df.drop(1000, inplace=True)
topRanksNow90_df.to_csv('out/current_rank.csv')
'''
pass

In [73]:
topRanksNow90_df = pd.read_csv('data/current_rank - changed.csv')
topRanksNow90_df

,userNum,nickname,gameId,seasonId,matchingMode,matchingTeamMode,characterNum,skinCode,characterLevel,gameRank,...,eventMissionResult.165,skillLevelInfo.1057440,skillLevelInfo.1057410,skillLevelInfo.1055320,skillLevelInfo.1058220,skillLevelInfo.1052410,skillLevelInfo.1026310,skillLevelInfo.1006310,skillLevelInfo.1055430,skillLevelInfo.1012310
0,725291,셀쓰녀가취향,22615555,15,3,3,39,1039001,18,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,725291,셀쓰녀가취향,22614952,15,3,3,39,1039001,16,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,725291,셀쓰녀가취향,22614596,15,3,3,46,1046000,10,6,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,725291,셀쓰녀가취향,22613843,15,3,3,46,1046000,16,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,1199780,탐피,22618328,15,3,3,2,1002002,15,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4753,3543478,CherryTree,22578049,15,3,3,1,1001003,17,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4754,3543478,CherryTree,22577461,15,3,3,1,1001003,17,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4755,3543478,CherryTree,22576966,15,3,3,1,1001003,16,3,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4756,3543478,CherryTree,22561824,15,3,3,1,1001003,17,1,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [127]:
skin = requests.get(url + 'data/CharacterSkin', headers=headers) # 스킨
time.sleep(1)
skin.json()

{'code': 200,
 'message': 'Success',
 'data': [{'name': 'Jackie',
   'code': 1001000,
   'characterCode': 1,
   'index': 0,
   'grade': 1,
   'purchaseType': 'FREE',
   'effectsPath': 'Effects/Jackie/S000',
   'projectilesPath': 'Projectiles/Jackie/S000',
   'objectPath': '',
   'fxSoundPath': 'Sound/FX/Jackie/S000',
   'voicePath': 'Sound/VOICE/Jackie/S000',
   'weaponMountPath': 'Weapons/Jackie/S000',
   'weaponMountCommonPath': 'Weapons/Common/C000',
   'indicatorPath': '',
   'projectilesDeflectorPath': ''},
  {'name': 'Executioner Jackie',
   'code': 1001001,
   'characterCode': 1,
   'index': 1,
   'grade': 2,
   'purchaseType': 'SHOP',
   'effectsPath': 'Effects/Jackie/S000',
   'projectilesPath': 'Projectiles/Jackie/S000',
   'objectPath': '',
   'fxSoundPath': 'Sound/FX/Jackie/S000',
   'voicePath': 'Sound/VOICE/Jackie/S000',
   'weaponMountPath': 'Weapons/Jackie/S001',
   'weaponMountCommonPath': 'Weapons/Common/C000',
   'indicatorPath': '',
   'projectilesDeflectorPath': ''

In [128]:
weapon = requests.get(url + 'data/CharacterAttributes', headers=headers)  # 무기
time.sleep(1)
weapon.json()

{'code': 200,
 'message': 'Success',
 'data': [{'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'OneHandSword',
   'controlDifficulty': 1,
   'attack': 4,
   'defense': 1,
   'disruptor': 1,
   'move': 4,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'TwoHandSword',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 2,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'Axe',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 1,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'DualSword',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 1,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Aya',
   'characterCode': 2,
   'mastery': 'Pistol',
   'controlDifficulty': 1,
   'attack': 3,
   'defense': 2,
   'disruptor': 2,
   'move': 3,
   'assistance': 1},
  {'charact

In [129]:
armor = requests.get(url + 'data/CharacterAttributes', headers=headers)  # 방어구
time.sleep(1)
armor.json()

{'code': 200,
 'message': 'Success',
 'data': [{'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'OneHandSword',
   'controlDifficulty': 1,
   'attack': 4,
   'defense': 1,
   'disruptor': 1,
   'move': 4,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'TwoHandSword',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 2,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'Axe',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 1,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Jackie',
   'characterCode': 1,
   'mastery': 'DualSword',
   'controlDifficulty': 1,
   'attack': 5,
   'defense': 1,
   'disruptor': 1,
   'move': 3,
   'assistance': 1},
  {'character': 'Aya',
   'characterCode': 2,
   'mastery': 'Pistol',
   'controlDifficulty': 1,
   'attack': 3,
   'defense': 2,
   'disruptor': 2,
   'move': 3,
   'assistance': 1},
  {'charact

In [126]:
metaType = requests.get(url + 'data/hash', headers=headers)  # 모든 metaType
time.sleep(1)
metaType.json()

{'code': 200,
 'message': 'Success',
 'data': {'Achievement': 25497290,
  'ActionCost': 3586852712,
  'ActionCost_Light': 2351618591,
  'Area': 993590539,
  'AreaAttributes': 3029790461,
  'AreaSound': 2271343438,
  'BattlePassMission': 3350932525,
  'BattlePassReward': 2212802826,
  'BattlePassRewardItem': 2758469976,
  'BattleZoneReward': 1214396460,
  'BattlefieldArea': 637359149,
  'Booster': 659324469,
  'BotAiModel': 1889052284,
  'BotCraft': 2283823037,
  'BotMastery': 1925639397,
  'BotNickName': 2061427971,
  'BotSkillBuild': 1316321053,
  'BulletCapacity': 2190560277,
  'BulletCapacity_Light': 1060615826,
  'Character': 44653840,
  'CharacterAttributes': 539867005,
  'CharacterExp': 4274863723,
  'CharacterLevelUpStat': 1117649331,
  'CharacterMastery': 3183597377,
  'CharacterModeModifier': 3266860007,
  'CharacterModeModifier_Light': 3169709200,
  'CharacterSkillVideos': 2536553328,
  'CharacterSkin': 1423839808,
  'CharacterState': 2180826605,
  'CharacterStateGroup': 9507

In [75]:
korean = requests.get(url + 'l10n/Korean', headers=headers)  # 게임 내부 정보의 한글 언어팩을 얻어온다.
time.sleep(1)
korean.json()

{'code': 200,
 'message': 'Success',
 'data': {'l10Path': 'https://d1wkxvul68bth9.cloudfront.net/l10n/l10n-Korean-20230106012136.txt'}}

In [120]:
krpack = requests.get('https://d1wkxvul68bth9.cloudfront.net/l10n/l10n-Korean-20230106012136.txt')
krpack.encoding='utf-8(BOM)'
with open('out/krpack.txt', 'w')as file:
    file.write(krpack.text)

In [121]:
test = requests.get(url + 'data/CharacterMastery', headers=headers)
time.sleep(1)
test.json()

{'code': 200,
 'message': 'Success',
 'data': [{'code': 1,
   'weapon1': 'OneHandSword',
   'weapon2': 'TwoHandSword',
   'weapon3': 'Axe',
   'weapon4': 'DualSword',
   'exploration1': 'Hunt',
   'exploration2': 'Craft',
   'exploration3': 'Search',
   'exploration4': 'Move',
   'survival1': 'Health',
   'survival2': 'Defense',
   'survival3': 'Trap',
   'survival4': 'None'},
  {'code': 2,
   'weapon1': 'Pistol',
   'weapon2': 'AssaultRifle',
   'weapon3': 'SniperRifle',
   'weapon4': 'None',
   'exploration1': 'Hunt',
   'exploration2': 'Craft',
   'exploration3': 'Search',
   'exploration4': 'Move',
   'survival1': 'Health',
   'survival2': 'Defense',
   'survival3': 'Trap',
   'survival4': 'None'},
  {'code': 3,
   'weapon1': 'Rapier',
   'weapon2': 'TwoHandSword',
   'weapon3': 'Spear',
   'weapon4': 'None',
   'exploration1': 'Hunt',
   'exploration2': 'Craft',
   'exploration3': 'Search',
   'exploration4': 'Move',
   'survival1': 'Health',
   'survival2': 'Defense',
   'surviva